In [1]:
import pandas as pd

/Users/stoneman/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/stoneman/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [16]:
df = pd.read_csv('/Users/stoneman/Library/CloudStorage/OneDrive-Vanderbilt/Social_Network_Analysis/Amazon_Recommandation_Project/data/train_and_test_index_features.csv')

In [17]:
df

,last_item_price,last_item_aggregated_price,last_second_price,last_second_aggregated_price,next_item_price,next_item_aggregated_price,next_item_index
0,9.48,9.250000,8.99,10.447500,8.99,9.423913,5284
1,3.60,5.801818,9.17,16.442222,5.25,8.673529,1836
2,58.91,31.204231,29.93,53.109016,32.49,32.914286,3412
3,3.89,7.948889,4.29,6.367500,6.39,3.752308,1313
4,7.99,1.247500,9.99,0.000000,-1.00,10.475000,1788
...,...,...,...,...,...,...,...
52194,19.99,17.590000,19.99,17.590000,1.20,2.094762,5306
52195,16.97,14.652353,13.99,12.578571,19.95,15.032105,2161
52196,4.88,10.776842,13.98,6.747647,1.50,7.539286,4518
52197,8.99,10.132857,9.99,9.212222,11.99,9.330000,1285


In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Splitting the data into features and targets
X = df.iloc[:, :4].values  # Features: columns 1 to 4
y = df.iloc[:, 4:7].values  # Targets: columns 5 and 6 (height and weight)

# Splitting the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

y_train = y_train[:, :2]  # Using only the height for this example

# Standardizing the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Converting numpy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Creating datasets and dataloaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [19]:
# Defining the neural network model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(4, 128)  # Input layer
        self.fc2 = nn.Linear(128, 64) # Second layer
        self.fc3 = nn.Linear(64, 32)  # Third layer
        self.fc4 = nn.Linear(32, 16)  # Fourth layer
        self.fc5 = nn.Linear(16, 2)   # Output layer

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.fc5(x)
        return x

model = Net()

# Defining the optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Training the model
epochs = 100
for epoch in range(epochs):
    model.train()
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}')
    
# Evaluating the model
model.eval()
test_loss = 0
with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        test_loss += criterion(output, target).item()

print(f'Test Loss: {test_loss / len(test_loader)}')


Epoch 1/100, Loss: 204.7728729248047
Epoch 2/100, Loss: 652.611572265625
Epoch 3/100, Loss: 955.0221557617188
Epoch 4/100, Loss: 181.2312774658203
Epoch 5/100, Loss: 503.7141418457031
Epoch 6/100, Loss: 629.7567749023438
Epoch 7/100, Loss: 211.3013153076172
Epoch 8/100, Loss: 309.2775573730469
Epoch 9/100, Loss: 329.45428466796875
Epoch 10/100, Loss: 375.2806091308594
Epoch 11/100, Loss: 153.23333740234375
Epoch 12/100, Loss: 3585.678955078125
Epoch 13/100, Loss: 265.96136474609375
Epoch 14/100, Loss: 214.0606689453125
Epoch 15/100, Loss: 324.3948059082031
Epoch 16/100, Loss: 236.7333526611328
Epoch 17/100, Loss: 202.76800537109375
Epoch 18/100, Loss: 190.6612091064453
Epoch 19/100, Loss: 390.03472900390625
Epoch 20/100, Loss: 681.1199340820312
Epoch 21/100, Loss: 558.0567016601562
Epoch 22/100, Loss: 390.3529968261719
Epoch 23/100, Loss: 1195.557861328125
Epoch 24/100, Loss: 446.397705078125
Epoch 25/100, Loss: 661.980712890625
Epoch 26/100, Loss: 7913.74462890625
Epoch 27/100, Loss: 

In [23]:
X_test

array([[29.99      , 20.45384615,  7.99      , 13.294     ],
       [59.99      , 88.19875   , 89.99      , 56.05666667],
       [11.        , 19.51      , 11.        , 19.51      ],
       ...,
       [ 7.99      ,  8.61896552,  8.98      ,  8.08642857],
       [83.18      , 77.94823529, 82.99      , 71.1332    ],
       [15.99      , 13.93736842, 13.99      , 15.20357143]])

In [27]:
y_train

array([[61.99      , 49.71066667],
       [ 4.3       ,  6.52928571],
       [11.        , 22.72789474],
       ...,
       [ 7.99      , 14.655     ],
       [19.99      , 20.84714286],
       [23.85      , 23.69555556]])

In [26]:
y_test

array([[  11.99      ,    9.70571429, 5167.        ],
       [  11.59      ,   11.09      , 1688.        ],
       [  10.        ,   10.52647059, 4063.        ],
       ...,
       [   8.99      ,   10.316     , 2094.        ],
       [  89.99      ,   85.243125  , 5393.        ],
       [  13.99      ,   14.2396875 , 6115.        ]])

In [30]:
items = pd.read_csv('/Users/stoneman/Library/CloudStorage/OneDrive-Vanderbilt/Social_Network_Analysis/Amazon_Recommandation_Project/data/products_train_UK_representations.csv')

In [35]:
items['aggregated_price'].describe()

count    6690.000000
mean       25.192863
std        62.290929
min        -1.000000
25%         8.766736
50%        13.685833
75%        23.374891
max      1684.555556
Name: aggregated_price, dtype: float64

In [37]:
items

,index,id,price,representation,aggregated_price
0,0,B07PNBPVN1,3.75,"[-0.21872417628765106, -0.2500726580619812, -0...",7.297000
1,1,B0B45XQ6Q3,8.59,"[-0.7130799889564514, -0.5445948839187622, -0....",11.618000
2,2,B01CR9IRWY,16.00,"[-0.8353576064109802, -0.6093693375587463, -0....",16.546154
3,3,B077RCBS9C,36.98,"[-0.8522624969482422, -0.5257986783981323, -0....",16.237500
4,4,B088BMDXPD,7.99,"[-0.8538389205932617, -0.47541171312332153, -0...",13.237500
...,...,...,...,...,...
6685,6685,B00NBR70DO,13.22,"[-0.9247933626174927, -0.5451830625534058, -0....",47.385000
6686,6686,B08HYPQ4VR,12.99,"[-0.9244785308837891, -0.5655009150505066, -0....",9.966667
6687,6687,B09LR8NRL9,2.89,"[-0.8537996411323547, -0.557623565196991, -0.9...",4.481667
6688,6688,B0987DMWKW,15.99,"[-0.6583928465843201, -0.3460875451564789, -0....",14.990000


In [41]:
import torch
from sklearn.metrics.pairwise import cosine_similarity


items_features = items[['price', 'aggregated_price']].values
items_features

array([[ 3.75      ,  7.297     ],
       [ 8.59      , 11.618     ],
       [16.        , 16.54615385],
       ...,
       [ 2.89      ,  4.48166667],
       [15.99      , 14.99      ],
       [15.99      , 15.99      ]])

In [48]:

acc_list = []

for i in range(len(X_test)):

    items_cos = items[['index', 'price', 'aggregated_price']].copy()

    # Generate predictions
    with torch.no_grad():
        predictions = model(torch.tensor(X_test[i], dtype=torch.float32))  # Or apply your model accordingly

    # Convert predictions to a numpy array if not already
    predictions = predictions.numpy()

    # Compute cosine similarity
    # Assuming 'output_vector' is a 1D numpy array containing the features you want to compare against
    cosine_similarities = cosine_similarity(predictions.reshape(1, -1), items_features)

    # Flatten the cosine similarity array and attach to DataFrame
    items_cos['similarity'] = cosine_similarities.flatten()

    # Sort by 'similarity' in descending order to get the most similar at the top
    top_100_items = items_cos.sort_values(by='similarity', ascending=False).head(100)

    # Retrieve the IDs
    top_100_indices = top_100_items['index'].tolist()

    # check if the item is in the top 100
    acc_list.append(1 if y_test[i][2] in top_100_indices else 0)


In [52]:
print(sum(acc_list)/len(acc_list)*100, '%')

2.5 %
